In [3]:
import sys
from pathlib import Path

# Notebook está em <root>/notebooks, então subimos um nível
PROJECT_ROOT = Path.cwd().parents[0]

# Garante que a raiz do projeto está no sys.path
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

PROJECT_ROOT


WindowsPath('c:/Enterprises/projects/ecommerce-demand-forecasting-3d-analytics')

In [4]:
from src.config import PROJECT_ROOT, RAW_DATA_DIR, PROCESSED_DATA_DIR

print(PROJECT_ROOT)
print(RAW_DATA_DIR)
print(PROCESSED_DATA_DIR)


C:\Enterprises\projects\ecommerce-demand-forecasting-3d-analytics
C:\Enterprises\projects\ecommerce-demand-forecasting-3d-analytics\data\raw
C:\Enterprises\projects\ecommerce-demand-forecasting-3d-analytics\data\processed


In [6]:
from src.data import load_raw_data, basic_cleaning, save_processed
